In [1]:
from openai import OpenAI
client = OpenAI()
import random
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd

import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
def split_into_sections(text, num_sentences_per_section=5):
    sentences = sent_tokenize(text)
    sections = []
    for i in range(0, len(sentences), num_sentences_per_section):
        section = ' '.join(sentences[i:i + num_sentences_per_section])
        sections.append(section)
    return sections


with open(r"C:\Users\jff35.DESKTOP-BPRRU50\Dropbox\Coding Dojo ABC Structure\P\Python\Python\Assignments\JanuaryMigrate\Notes\tsundere.txt", "r") as file:
    biography_text = file.read()
    
sections = split_into_sections(biography_text, num_sentences_per_section=2)


def get_embedding(text, model="text-embedding-ada-002"):
	text = text.replace("\n", " ")
	return client.embeddings.create(input = [text], model=model).data[0].embedding	

def calculate_similarity(emb1, emb2):
    # Convert lists to numpy arrays and reshape them to 2D arrays
    emb1_array = np.array(emb1).reshape(1, -1)
    emb2_array = np.array(emb2).reshape(1, -1)
    return cosine_similarity(emb1_array, emb2_array)[0][0]

similarity_threshold = 0.5


question = "What is TsundereChatGPT-chan's favorite color?"
question_embedding = get_embedding(question)
data = { "section_text": sections,
		"embedding": [get_embedding(section_text) for section_text in sections]}
df = pd.DataFrame(data)
df["similarity"] = df["embedding"].apply(lambda emb: calculate_similarity(emb, question_embedding))
relevant_sections = df[df["similarity"] > similarity_threshold]["section_text"]
context = ' '.join(relevant_sections.tolist())


message = {
    "role": "user",
    "content": f"{context}\n\nQuestion: {question}"
}


character_response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[message])
print_response = character_response.choices[0].message.content
print(print_response)

[nltk_data] Downloading package punkt to C:\Users\jff35.DESKTOP-
[nltk_data]     BPRRU50\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TsundereChatGPT-chan's favorite color is aquamarine.
